### Overriding Methods

In [1]:
class Bird:
    def __init__(self):
        self.hungry = True
    
    def eat(self):
        if self.hungry:
            print('Nom nom nom')
            self.hungry = False
        else:
            print('Full, thanks')

In [2]:
b = Bird()
b.eat()

Nom nom nom


In [3]:
b.eat()

Full, thanks


In [4]:
class SongBird(Bird):
    def __init__(self):
        self.sound = 'Squawk!'
        
    def sing(self):
        print(self.sound)

In [5]:
sb = SongBird()
sb.sing()

Squawk!


In [6]:
sb.eat()

AttributeError: 'SongBird' object has no attribute 'hungry'

In [7]:
class SongBird(Bird):
    def __init__(self):
        Bird.__init__(self)
        self.sound = 'Warble warble'
    
    def sing(self):
        print(self.sound)

In [8]:
sb = SongBird()
sb.sing()

Warble warble


In [9]:
sb.eat()

Nom nom nom


In [10]:
sb.eat()

Full, thanks


In [11]:
class SongBird(Bird):
    def __init__(self):
        super(SongBird, self).__init__()
        self.sound = 'Cackaw caw'

    def sing(self):
        print(self.sound)

In [12]:
sb = SongBird()
sb.sing()

Cackaw caw


In [13]:
sb.eat()
sb.eat()

Nom nom nom
Full, thanks


# Item Access

In [21]:
def check_index(key):
    if not isinstance(key, int): raise TypeError
    if key < 0: raise IndexError

In [22]:
class ArithmeticSequence:
    def __init__(self, start=0, step=1):
        self.start = start
        self.step = step
        self.changed = {}
        
    def __getitem__(self, key):
        check_index(key)
        try: return self.changed[key]
        except KeyError: return self.start + key * self.step
        
    def __setitem__(self, key, value):
        check_index(key)
        self.changed[key] = value

In [23]:
s = ArithmeticSequence(1, 2)

In [24]:
s[4]

9

In [25]:
s[3]

7

In [27]:
s[3] = 3

In [28]:
s[3]

3

In [29]:
s[4]

9

### Subclassing `list`, `dict`, and `str`

In [1]:
class CounterList(list):
    def __init__(self, *args):
        super(CounterList, self).__init__(*args)
        self.counter = 0
        
    def __getitem__(self, index):
        self.counter += 1
        return super(CounterList, self).__getitem__(index)

In [2]:
cl = CounterList(range(10))
cl

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
cl.reverse()
cl

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [4]:
cl.counter

0

In [5]:
cl[4] + cl[2]

12

In [6]:
cl.counter

2

In [9]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    
    def setSize(self, size):
        self.width, self.height = size
        
    def getSize(self):
        return self.width, self.height

In [10]:
r1 = Rectangle()
r1.width = 10
r1.height = 5
r1.getSize()

(10, 5)

In [11]:
r1.setSize((20, 10))
r1.getSize()

(20, 10)

### Static and Class Methods

In [13]:
class MyClass:
    def static_method():
        print('This is a static method')
    
    static_method = staticmethod(static_method)
    
    def class_method(cls):
        print('This is a class method of', cls)
    
    class_method = classmethod(class_method)

In [14]:
# Equivalently
class MyClass:
    @staticmethod
    def static_method():
        print('This is a static method')
    
    @classmethod
    def class_method(cls):
        print('This is a class method of', cls)

In [15]:
MyClass.static_method()

This is a static method


In [16]:
MyClass.class_method()

This is a class method of <class '__main__.MyClass'>


### `__getattr__`, `__setattr__`, and Friends

In [17]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
        
    def __setattr__(self, name, value):
        if name == 'size':
            self.width, self.height = value
        else:
            self.__dict__[name] = value
            
    def __getattr__(self, name):
        if name == 'size':
            return self.width, self.height
        else:
            raise AttributeError

# Iterators

In [24]:
class Fibonacci:
    def __init__(self):
        self.a = 0
        self.b = 1
        
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a
    
    def __iter__(self):
        return self

In [26]:
fibs1 = Fibonacci()

# Find first Fibonacci number > 1000
for f in fibs1:
    if f > 1000:
        print(f)
        break

1597


In [29]:
fibs2 = Fibonacci()
next(fibs2)

1

In [30]:
next(fibs2)

1

In [31]:
next(fibs2)

2

### Making Sequences from Iterators

In [2]:
class TestIterator:
    value = 0
    
    def __next__(self):
        self.value +=1
        if self.value > 10:
            raise StopIteration
        return self.value
    
    def __iter__(self):
        return self

In [3]:
ti = TestIterator()
print(list(ti))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Generators

In [5]:
nested = [[1, 2], [3, 4], [5]]

def flatten(nested):
    for sublist in nested:
        for element in sublist:
            yield element

In [9]:
for num in flatten(nested):
    print(num, end='---')

1---2---3---4---5---

In [10]:
list(flatten(nested))

[1, 2, 3, 4, 5]

In [12]:
def flatten(nested):
    try:
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

In [13]:
list(flatten([[[1], 2], 3, 4, [5, [6, 7]], 8]))

[1, 2, 3, 4, 5, 6, 7, 8]

In [14]:
# Account for strs
def flatten(nested):
    try:
        # Don't iterate over strs
        try:
            nested + ''
        except TypeError:
            pass
        else:
            raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

In [16]:
list(flatten(['foo', ['bar', ['fubar']]]))

['foo', 'bar', 'fubar']

### Generator Methods

In [17]:
def repeater(value):
    while True:
        new = (yield value)
        if new is not None:
            value = new

In [19]:
r = repeater(42)
print(next(r))
print(next(r))

42
42


In [20]:
r.send('Hello')
print(next(r))
print(next(r))

Hello
Hello
